In [8]:
!pip install pandas
!pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import pandas as pd
import mysql.connector

In [2]:
df = pd.read_csv('../data/raw/ISCED_2013.csv', encoding ='latin-1', delimiter=';')

In [3]:
df.head()

,COD_RAMA,NOM_RAMA,COD_RAMA_N2,NOM_RAMA_N2,COD_RAMA_N3,NOM_RAMA_N3,COD_RAMA_N4,NOM_RAMA_N4,COD_RAMA_N5,NOM_RAMA_N5
0,1,Artes y humanidades,1,Educación,11,Educación,111,Ciencias de la educación,11101,Pedagogía
1,1,Artes y humanidades,1,Educación,11,Educación,111,Ciencias de la educación,11102,Didácticas específicas
2,1,Artes y humanidades,1,Educación,11,Educación,112,Formación de docentes de enseñanza infantil,11201,Educación infantil
3,1,Artes y humanidades,1,Educación,11,Educación,113,Formación de docentes de enseñanza primaria,11301,Educación primaria
4,1,Artes y humanidades,1,Educación,11,Educación,114,Formación de docentes de educación secundaria ...,11401,Otros maestros


In [4]:
columns = df.columns
new_columns_dict = {}
cod_count = 1
nom_count = 1

for col in columns:
    if 'COD_' in col:
        new_columns_dict[col] = f'codigo_rama_{cod_count}'
        cod_count += 1
    elif 'NOM_' in col:
        new_columns_dict[col] = f'nombre_rama_{nom_count}'
        nom_count += 1
    else:
        new_columns_dict[col] = col

# Use the rename() method to rename the columns
df.rename(columns=new_columns_dict, inplace=True)

# Print the updated DataFrame
df.head(2)

,codigo_rama_1,nombre_rama_1,codigo_rama_2,nombre_rama_2,codigo_rama_3,nombre_rama_3,codigo_rama_4,nombre_rama_4,codigo_rama_5,nombre_rama_5
0,1,Artes y humanidades,1,Educación,11,Educación,111,Ciencias de la educación,11101,Pedagogía
1,1,Artes y humanidades,1,Educación,11,Educación,111,Ciencias de la educación,11102,Didácticas específicas


In [7]:
cnx =mysql.connector.connect(
    host='localhost',
    user='admin',
    password='admin',
    database='dw',
    auth_plugin='mysql_native_password'
)

cursor = cnx.cursor()

In [ ]:
data = [tuple(row) for _, row in df.iterrows()]

query=f"INSERT INTO stage_ramas_conocimiento VALUES ({','.join(['%s'] * len(df.columns))})"

cursor.executemany(query, data)

try:
    cnx.commit()
    print("Datos insertados de manera exitosa!")
except Exception as e:
    print("Error al insertar los datos")
    print("Error code:", e.errno)
    print("Error description:", e.strerror)

In [9]:
query=f"SELECT * FROM stage_ramas_conocimiento"

df_2 = pd.read_sql(query, cnx)

df_2.head(2)

/tmp/ipykernel_15271/2414799872.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_2 = pd.read_sql(query, cnx)


,codigo_rama_1,nombre_rama_1,codigo_rama_2,nombre_rama_2,codigo_rama_3,nombre_rama_3,codigo_rama_4,nombre_rama_4,codigo_rama_5,nombre_rama_5
0,1,Artes y humanidades,1,Educación,11,Educación,111,Ciencias de la educación,11101,Pedagogía
1,1,Artes y humanidades,1,Educación,11,Educación,111,Ciencias de la educación,11102,Didácticas específicas


In [ ]:
cursor.close()
cnx.close()

In [ ]:
#mysql -u admin -p